In [ ]:
import os
import sys
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv(os.path.expanduser('~/dropbox/ecg-ici/explore/cv_event_by_case/tensors_union.csv')).dropna()
df['cv_event'] = df['ici_cv_event_180_days_post_ecg']
df['case'] = df['ici_case_180_days_post_ecg_case']

In [ ]:
case = df[df['case'] == 1]
control = df[df['case'] == 0]

In [ ]:
_case_event = case[case['cv_event'] == 1]
_case_no_event = case[case['cv_event'] == 0]

_control_event = control[control['cv_event'] == 1]
_control_no_event = control[control['cv_event'] == 0]

In [ ]:
train_ratio = 0.6
valid_ratio = 0.2
test_ratio = 0.2

In [ ]:
bootstrap_folder = os.path.expanduser('~/dropbox/ecg-ici/data/bootstraps')
os.makedirs(bootstrap_folder, exist_ok=True)
cols = ['patientid', 'case', 'cv_event']
for i in range(10):
    case_event = _case_event.sample(frac=1, random_state=i)
    case_no_event = _case_no_event.sample(frac=1, random_state=i)
    control_event = _control_event.sample(frac=1, random_state=i)
    control_no_event = _control_no_event.sample(frac=1, random_state=i)

    cae_tv_idx = int(len(case_event) * train_ratio)
    cae_vt_idx = int(len(case_event) * (train_ratio + valid_ratio)) + 1

    cane_tv_idx = int(len(case_no_event) * train_ratio)
    cane_vt_idx = int(len(case_no_event) * (train_ratio + valid_ratio)) + 1

    coe_tv_idx = int(len(control_event) * train_ratio)
    coe_vt_idx = int(len(control_event) * (train_ratio + valid_ratio)) + 1

    cone_tv_idx = int(len(control_no_event) * train_ratio)
    cone_vt_idx = int(len(control_no_event) * (train_ratio + valid_ratio)) + 1

    train = pd.concat([case_event[:cae_tv_idx], case_no_event[:cane_tv_idx], control_event[:coe_tv_idx], control_no_event[:cone_tv_idx]])
    valid = pd.concat([case_event[cae_tv_idx:cae_vt_idx], case_no_event[cane_tv_idx:cane_vt_idx], control_event[coe_tv_idx:coe_vt_idx], control_no_event[cone_tv_idx:cone_vt_idx]])
    test = pd.concat([case_event[cae_vt_idx:], case_no_event[cane_vt_idx:], control_event[coe_vt_idx:], control_no_event[cone_vt_idx:]])

    os.makedirs(f'{bootstrap_folder}/{i}', exist_ok=True)
    train[cols].sample(frac=1, random_state=i).to_csv(f'{bootstrap_folder}/{i}/train.csv', index=False)
    valid[cols].sample(frac=1, random_state=i).to_csv(f'{bootstrap_folder}/{i}/valid.csv', index=False)
    test[cols].sample(frac=1, random_state=i).to_csv(f'{bootstrap_folder}/{i}/test.csv', index=False)